# Datacraft Events Assistant – Querying a Local Database with Natural Language

Welcome to this hands-on notebook!  
The goal is to build an intelligent assistant that can answer questions like (using a database of events extracted from Datacraft’s internal calendar):

> *"What are the upcoming events in French?"*  
> *"How many workshops are related to AI?"*

All of that by querying a **local database** using a **natural language interface** powered by an LLM (Large Language Model).


## What You'll Learn

In this notebook, you will:

Connect a natural language agent to a local SQLite database  
Define a tool that lets the agent run SQL queries  
Ask real questions and get structured answers

You’ll also learn how to use the [`smolagents`](https://github.com/smol-ai/smolagent) framework to combine tools with LLMs like GPT-4.


## The `events` Database

The agent will query a local database of events extracted from Datacraft’s internal calendar.  
The database is a simple SQLite file called `events.db`, and it contains a single table: **`evenements`**.

Each row represents one event (a workshop, talk, or training session).

### Main columns:

| Column         | Description                                        |
|----------------|----------------------------------------------------|
| `nomenclature` | Unique code for the event (e.g. `0706-AI-Intro`)   |
| `titre`        | Event title (e.g. `"Intro to Generative AI"`)      |
| `description`  | Full description of the event                      |
| `date`         | Date of the event (as text, e.g. `2024-06-03`)     |
| `horaire`      | Time range (e.g. `"9h00 - 17h30"`)                 |
| `tag`          | Topics or tags (e.g. `#AI`, `#DataGouv`)           |
| `langue`       | Language of the event (e.g. `"français"`, `"english"`) |


## Technologies Used

This notebook uses:

- `smolagent` – to manage the agent and tools
- `OpenAI` – to interpret your questions
- `SQLAlchemy` – to connect to the database


## Expected Outcome

By the end of this notebook, you’ll be able to write:

```python
agent.run("List all French events after July 1st.")

In [6]:
import os
from smolagents import tool, CodeAgent, OpenAIServerModel
from dotenv import load_dotenv
from sqlalchemy import create_engine, text, inspect

load_dotenv(override=True)

True

## Getting more info about the database column names

In [ ]:
engine = create_engine("sqlite:///../../data/events.db")
inspector = inspect(engine)
for col_info in inspector.get_columns("evenements"):
    print(col_info["name"])

nomenclature
titre
description
date
horaire
tag
langue


### Defining a Tool for the Agent: `query_events_db`

In `smolagent`, a **tool** is a Python function that the agent can call to interact with the outside world — such as querying a database, calling an API, reading a file, or performing calculations.

#### Why define a tool?

LLMs like GPT-4 don't have direct access to your local environment or files.  
By wrapping a function with the `@tool` decorator, you allow the agent to **"see" and use** that function when it decides it's relevant to the user’s question.


#### What does a tool need?

A tool in `smolagent` is simply a Python function, but it must follow these **three key rules**:

1. **A clear and unique name**  
   The name of the function is how the agent will refer to it internally. It should describe the action it performs.

2. **A well-written docstring**  
   The agent reads the docstring to understand:
   - what the tool does
   - how to use it
   - what arguments it accepts
   - what it returns  
   
   Without a good docstring, the agent will either ignore the tool or misuse it.

3. **A simple and reliable interface**  
   The tool should expect only standard Python types as arguments (`str`, `int`, `float`, etc.) and return a string or JSON-serializable output.

#### Our example: `query_events_db`

This tool allows the agent to send **raw SQL queries** to the `evenements` table in our local database.

You can ask the agent:
- `"What are the events in French?"`
- `"List all events that mention AI."`
- `"How many events are scheduled after July?"`

The agent will translate your question into SQL, run the query using this tool, and return the results in plain text.

In [8]:
@tool
def query_events_db(query: str) -> str:
    """
    Allows querying the `evenements` table, which contains cleaned information about workshops and events.

    Table: evenements

    Available columns include:
    - nomenclature: unique identifier or code for the event (e.g. "0706-MiniFormation-Capdigital")
    - titre: title of the event (e.g. "Introduction to Generative AI")
    - description: detailed description of the event
    - date: date of the event, stored as text (e.g. "2024-06-03")
    - horaire: time slot or duration of the event (e.g. "9h00 - 17h30")
    - tag: optional topic tags (e.g. "#DataGouv", "#IA")
    - langue: language of the event (e.g. "français", "anglais")

    You can write SQL queries such as:
    - SELECT titre FROM evenements WHERE langue = 'français';
    - SELECT COUNT(*) FROM evenements WHERE tag LIKE '%IA%';
    - SELECT DISTINCT langue FROM evenements;
    - SELECT * FROM evenements ORDER BY date DESC;

    Note: the `date` column is stored as text. If you need to sort or filter by date,
    you can use SQL date functions (e.g. `STRFTIME`) or cast the column as needed.

    Args:
        query: A valid SQL query string to execute on the `evenements` table.

    Returns:
        A string containing the query results in plain text format.
    """
    engine = create_engine("sqlite:///../../data/events.db")
    output = ""
    with engine.connect() as con:
        rows = con.execute(text(query))
        for row in rows:
            output += "\n" + str(row)
    return output

We now set up the model that will power the agent.
We’re using OpenAI’s API to access the model gpt-4o.

In [9]:
model = OpenAIServerModel(
    model_id="gpt-4o",
    api_base="https://api.openai.com/v1",
    api_key=os.environ["OPENAI_API_KEY"],
)


Now that we have:

- a model (`gpt-4o` via OpenAI),
- a tool (`query_events_db`) that knows how to query the event database,  

we can combine them into a single `CodeAgent`.

The `CodeAgent` is the core component of the `smolagent` framework.  
It takes a list of tools and a language model, and automatically decides:

- When to call a tool
- How to use it
- What answer to return to the user


In [10]:
agent = CodeAgent(
    tools=[query_events_db],
    model=model,
)

let's try it ! 

In [13]:
question = "What are the different types of events organized?"
agent.run(question)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What are the different types of events organized?                                                               │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt-4o ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  query_result = query_events_db("SELECT DISTINCT titre FROM evenements;")                                         
  print(query_result)                                                                                              
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:

('Ne pas supprimer - Utile pour dev base',)
('AI Fairness et biais des algorithmes',)
('NLP - Sentiment Analysis - Few Shot Learning',)
('S’il vous plait… dessine-moi une explication – Partie 1',)
('S’il vous plait… dessine-moi une explication – Partie 2',)
('S’il vous plait… dessine-moi une explication – Partie 3',)
('Données Twitter - Analyse de la viralité et propagation d’une fake news',)
('Industrialisation de votre app data science : quand la baleine part dans les nuages !',)
('Les talents autistes ou Asperger dans la Data : Mieux les comprendre, les recruter et les intégrer dans ses 
équipes',)
('Comment insuffler une culture Data dans son organisation ?',)
('Mindshake Time : NLP - Few Shot Learning - Sentiment Analysis',)
('Multivariate and functional anomaly detection - Partie 1',)
('Multivariate and Functional Anomaly Detection – Partie 2',)
('Multivariate and Functional Anomaly Detection – Partie 3',)
('Associations causales dans les tweets liés au diabète',)
('Données synthétiques : Méthodes de génération et limites',)
("Détection d'anomalies à partir de données structurées - Application à la détection d'intrusion",)
("Meet-Up Paris Data Ladies X datacraft - L'IA pour ou contre le climat",)
('Interprétabilité - Shapley dans tous ses états !',)
("Le jeu de l'IA@",)
("L'Intelligence Artificielle au service des Ressources Humaines",)
('Few shot Learning NLP ',)
("Le NLP appliqué à l'extraction et la structuration des compétences pour la gestion des carrières",)
('Pandas : Des bonnes pratiques',)
('Animer sa communauté data',)
('Quel est le meilleur système ? Évaluer les réseaux pré-entraînés',)
('Keakr – Système de recommandation de musique urbaine',)
("Les défis pratiques de l'IA explicable : application aux services financiers",)
('Est-ce que cet algorithme me trompe ? Comment constater, expliquer et prouver les biais ?',)
('Est-ce que cet algorithme me trompe ?',)
('Comment constater, expliquer et prouver les biais ? ',)
("Identifier des réponses atypiques à l'entraînement de cyclistes de haut niveau",)
('Masterclass by Erwin Logt, Danone CIO, for Sorbonne University PHD students',)
('Kubernetes',)
('A Framework to Learn with Interpretation',)
("Quels éléments prendre en compte pour calculer le ROI d'un projet data ?",)
('Forecasting Multi-Horizon : prédire des ventes à différents horizons',)
('Vision Transformer classification applied to computer vision medical diagnosis',)
('Données Géospatiales : Dataviz avec Plotly en mode BYOD* !',)
('Peut-on combiner analyse logique et Machine Learning ?',)
('Concours datacraft, venez tester l’escape game digital !',)
('Dataviz - Utiliser Dash pour la création de dashboards interactifs ',)
('Rencontres Data Gouvernance #1 - Gérer ses données comme un produit - Bonne pratique de la gouvernance des 
données',)
('Détection de contours et annotation automatique d’images de fonds marins, c’est quoi les meilleures méthodes ?',)
("Rencontres Data Gouvernance #2 - S'organiser en Data Domains",)
('Visualisation de données géospatiales : bonnes pratiques appliquées aux feux de forêt',)
("IA responsable et de confiance : retour d'expériences sur la certification et la labellisation.",)
('Atténuation des biais de genre : application à la reconnaissance faciale',)
("IA et Santé : Prédiction du pH du fœtus lors de l'accouchement",)
('A la découverte de Julia !',)
("Data & Jeu Vidéo - Optimiser l'expérience joueur grâce aux données",)
("Nouveaux outils pour l'apprentissage non-supervisé",)
("Apprentissage non supervisé appliqué à l'analyse des comportements clients",)
('Comment exploiter les informations postées sur les réseaux sociaux',)
("Pourquoi s'intéresser à l'IA hybride ?",)
('Mentoring - open-source and scikit-learn',)
("Les apports de l'approche causale : exploration sur des cas d'usage concrets",)
('Causal Discovery : partons à la découverte du graphe causal',)
("Qualité des soins en maison de retraite : l'IA peut-elle permettre un meilleur suivi ?",)
("Auditer se

[Step 1: Duration 2.18 seconds| Input tokens: 2,204 | Output tokens: 82]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer([                                                                                                   
      "Forecasting", "Fairness", "VideoGame", "UnsupervisedLearning",                                              
      "Causality", "Health", "Climate", "Annotation", "Robotics",                                                  
      "DataQuality", "GAN", "DataPipeline", "Sustainability", "Docker",                                            
      "Kubernetes", "OpenData", "Pandas", "DataDomains", "SustainableAI",                                          
      "Trust", "RStats", "ChatGPT", "Scraping", "AnomalyDetection",                                                
      "Deepfake", "ComputerVision", "AIrisks", "LoRA",                                                             
      "GeometricDeepLearning", "DataCatalog", "MachineLearning", "AIEthics",                                       
      "BayesianInference", "FrugalAI", "GenerativeAI", "TrustworthyAI",                                            
      "MLOps", "Evaluation", "Forensics", "Fraud", "OSINT",                                                        
      "ResponsibleAI", "Transformers", "Agents", "DeveloperExperience",                                            
      "Python", "BestPractices", "Hackathon", "ExplainableAI",                                                     
      "ConformalPrediction", "DigitalTwin"                                                                         
  ])                                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: ['Forecasting', 'Fairness', 'VideoGame', 'UnsupervisedLearning', 'Causality', 'Health', 'Climate', 
'Annotation', 'Robotics', 'DataQuality', 'GAN', 'DataPipeline', 'Sustainability', 'Docker', 'Kubernetes', 
'OpenData', 'Pandas', 'DataDomains', 'SustainableAI', 'Trust', 'RStats', 'ChatGPT', 'Scraping', 'AnomalyDetection',
'Deepfake', 'ComputerVision', 'AIrisks', 'LoRA', 'GeometricDeepLearning', 'DataCatalog', 'MachineLearning', 
'AIEthics', 'BayesianInference', 'FrugalAI', 'GenerativeAI', 'TrustworthyAI', 'MLOps', 'Evaluation', 'Forensics', 
'Fraud', 'OSINT', 'ResponsibleAI', 'Transformers', 'Agents', 'DeveloperExperience', 'Python', 'BestPractices', 
'Hackathon', 'ExplainableAI', 'ConformalPrediction', 'DigitalTwin']

[Step 3: Duration 4.03 seconds| Input tokens: 13,347 | Output tokens: 476]

['Forecasting',
 'Fairness',
 'VideoGame',
 'UnsupervisedLearning',
 'Causality',
 'Health',
 'Climate',
 'Annotation',
 'Robotics',
 'DataQuality',
 'GAN',
 'DataPipeline',
 'Sustainability',
 'Docker',
 'Kubernetes',
 'OpenData',
 'Pandas',
 'DataDomains',
 'SustainableAI',
 'Trust',
 'RStats',
 'ChatGPT',
 'Scraping',
 'AnomalyDetection',
 'Deepfake',
 'ComputerVision',
 'AIrisks',
 'LoRA',
 'GeometricDeepLearning',
 'DataCatalog',
 'MachineLearning',
 'AIEthics',
 'BayesianInference',
 'FrugalAI',
 'GenerativeAI',
 'TrustworthyAI',
 'MLOps',
 'Evaluation',
 'Forensics',
 'Fraud',
 'OSINT',
 'ResponsibleAI',
 'Transformers',
 'Agents',
 'DeveloperExperience',
 'Python',
 'BestPractices',
 'Hackathon',
 'ExplainableAI',
 'ConformalPrediction',
 'DigitalTwin']